# PCR Implementation

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import r2_score
from sklearn.base import BaseEstimator
import scipy.linalg
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import cross_val_score

In [ ]:
class PCR(BaseEstimator):
    
    def __init__(self, n_components=1):
        self.n_components = n_components
    
    def fit(self, X, y):
        tr = TruncatedSVD(n_components=self.n_components).fit(X)
        X = tr.transform(X)
        X = tr.inverse_transform(X)
        self.coef_ = scipy.linalg.pinv(X.T @ X) @ X.T @ y
        return self

    def predict(self, X):
        return X @ self.coef_

# Simple: Prediction from Noisy Low-Rank Measurements

In [ ]:
n = 1000
T = 5000
u = np.random.normal(0, 1, size=(n, 1))
v = np.random.normal(0, 1, size=(T, 1))
W = u @ v.T
Z = W + np.random.normal(0, .7, size=(n, T))

In [ ]:
X = Z[1:, :-10].T
y = Z[0, :-10]
Xtest = Z[1:, -10:].T
ytest = Z[0, -10:]
gtest = W[0, -10:]

In [ ]:
cross_val_score(PCR(), X, y, scoring='r2')

In [ ]:
est = PCR().fit(X, y)

In [ ]:
pred_test = est.predict(Xtest)

In [ ]:
plt.plot(pred_test, label='pred')
plt.plot(ytest, label='Y')
plt.plot(gtest, label='E[Y]')
plt.legend()
plt.show()

In [ ]:
plt.hist(est.coef_)
plt.show()

# Synthetic Controls with Staggered Rollout

In [ ]:
n = 1000 # n units
T = 5000 # n overall time periods
K = 2 # n actions (for now has to be true)
u = np.random.normal(0, 1, size=(n, 1)) # unit latent factors
v = np.random.normal(0, 1, size=(K, T, 1)) # (action, time) latent factors
W = np.einsum('ij,jtk->itk', u, v.T) # true mean potential outcomes for each unit and period
Z = W + np.random.normal(0, 1, size=(n, T, K)) # random potential outcomes for each unit and period

In [ ]:
T0 = T - 10 # pre-treatment period length
t0 = np.random.choice(np.arange(T0, T + 1), size=n, replace=True) # choose random rollout time after T0
time = np.tile(np.arange(T), (n, 1)) # helper matrix
D = (time >= np.tile(t0.reshape(-1, 1), (1, T))) * 1 # set treatment to 1 after rollout

In [ ]:
Zobs = Z[:, :, 0] * (1 - D) + Z[:, :, 1] * D # observed noisy outcomes
Wobs = W[:, :, 0] * (1 - D) + W[:, :, 1] * D # observed true mean outcomes

In [ ]:
# we only care about the first unit and we use the rest to predict
X = Zobs[1:, :T0].T
y = Zobs[0, :T0]
Xtest = Zobs[1:, T0:].T
ytest = Zobs[0, T0:]

In [ ]:
t0[0] - T0

In [ ]:
pred_test = np.zeros(T)
for t in np.arange(T0, T):
    donors = (D[1:, t] == 0) # find units that are un-treated in this post-treatment period
    est = PCR().fit(X[:, donors], y) # find coefficients to donor units using PCR
    pred_test[t] = est.predict(Xtest[t - T0, donors]) # predict the outcome for the target unit for this period

In [ ]:
t1 = T0
t2 = T
plt.plot(pred_test[t1:t2], label='pred')
# plt.plot(ytest, label='observed')
plt.plot(Wobs[0, t1:t2], label='E[Y]')
plt.plot(W[0, t1:t2, 0], label='E[Y(0)]')
plt.axvline(t0[0] - T0 - 1, color='magenta', linestyle='--')
plt.xticks(ticks=np.arange(t2 - t1), labels=D[0, t1:t2])
plt.axvline(t0[0] - T0 - 1, color='magenta', linestyle='--')
plt.xlabel('treatment per period')
plt.legend()
plt.show()

# Synthetic Interventions

In [ ]:
n = 1000 # n units
T = 5000 # n overall time periods
K = 3 # n actions (for now has to be true)
u = np.random.normal(0, 1, size=(n, 1)) # unit latent factors
v = np.random.normal(0, 1, size=(K, T, 1)) # (action, time) latent factors
W = np.einsum('ij,jtk->itk', u, v.T) # true mean potential outcomes for each unit and period
Z = W + np.random.normal(0, 1, size=(n, T, K)) # random potential outcomes for each unit and period

In [ ]:
T0 = T - 10 # pre-treatment period length
t0 = np.random.choice(np.arange(T0, T + 1), size=n, replace=True) # choose random rollout time after T0
time = np.tile(np.arange(T), (n, 1)) # helper matrix
D = np.random.choice(np.arange(1, K), size=(n, T), replace=True)
D = (time >= np.tile(t0.reshape(-1, 1), (1, T))) * D # set treatment to 1 after rollout

In [ ]:
Zobs = Z[:, :, 0] * (D == 0) # building observed noisy outcomes
Wobs = W[:, :, 0] * (D == 0) # building observed true mean outcomes
for t in np.arange(1, K):
    Zobs += Z[:, :, t] * (D == t) 
    Wobs += W[:, :, t] * (D == t)

In [ ]:
# we only care about the first unit and we use the rest to predict
X = Zobs[1:, :T0].T
y = Zobs[0, :T0]
Xtest = Zobs[1:, T0:].T
ytest = Zobs[0, T0:]

In [ ]:
t0[0] - T0

In [ ]:
# calculate mean counterfactual outcome for each period and each potential treatment
pred_test = np.zeros((K, T))
for k in np.arange(K):
    for t in np.arange(T0, T):
        donors = (D[1:, t] == k) # find units that received treatment k in this post-treatment period
        est = PCR().fit(X[:, donors], y) # find coefficients to donor units using PCR
        pred_test[k, t] = est.predict(Xtest[t - T0, donors]) # predict the outcome for the target unit for this period

In [ ]:
t1 = T0
t2 = T
for k in range(K):
    plt.figure(figsize=(15, 5))
    # plt.plot(ytest, label='observed')
    plt.plot(Wobs[0, t1:t2], label='E[Y]')
    plt.plot(pred_test[k, t1:t2], label=f'pred({k})')
    plt.plot(W[0, t1:t2, k], label=f'E[Y({k})]')
    plt.xticks(ticks=np.arange(t2 - t1), labels=D[0, t1:t2])
    plt.axvline(t0[0] - T0 - 1, color='magenta', linestyle='--')
    plt.xlabel('treatment per period')
    plt.legend()
    plt.show()

# Synthetic Blips

In [ ]:
n = 1000 # n units
T = 5000 # n overall time periods
K = 3 # n actions (for now has to be true)
lags = 2 # number of lags that impact current outcome
u = np.random.normal(0, 1, size=(n, 1)) # unit latent factors
v = np.random.normal(0, 1, size=(K, lags, T, 1)) # (action, time) latent factors
W = np.einsum('ij,jltk->iltk', u, v.T) # true mean potential blips for each unit and period and lag
Z = W + np.random.normal(0, 1, size=(n, T, lags, K)) # random potential blips for each unit, period and lag

In [ ]:
T0 = T - 10 # pre-treatment period length
t0 = np.random.choice(np.arange(T0, T + 1), size=n, replace=True) # choose random rollout time after T0
time = np.tile(np.arange(T), (n, 1)) # helper matrix
D = np.random.choice(np.arange(1, K), size=(n, T), replace=True)
D = (time >= np.tile(t0.reshape(-1, 1), (1, T))) * D # set treatment to 1 after rollout

In [ ]:
Zobs = np.zeros(Z.shape[:2]) # building observed noisy outcomes
Wobs = np.zeros(W.shape[:2]) # building observed true mean outcomes
for ell in range(lags): # for each lag period
    Dell = np.roll(D, ell) # we find the lag treatment for each period
    Dell[:, :ell] = 0
    for k in range(K):
        Zobs += Z[:, :, ell, k] * (Dell == k) # we add the lag blip effect of that lag treatment
        Wobs += W[:, :, ell, k] * (Dell == k) # we add the lag blip effect of that lag treatment

In [ ]:
t0[0] - T0

In [ ]:
from joblib import Parallel, delayed

def donor_weights(i):
    X = Zobs[:, :T0].T
    y = Zobs[i, :T0]
    # calculate mean counterfactual outcome for each period and each potential treatment
    Beta = np.zeros((K, T - T0, n))
    for k in np.arange(K):
        for t in np.arange(T0, T):
            # find units that received treatment k in period t, as their first treatment in this post-treatment period
            donors = (D[:, t] == k) & np.all(D[:, :t] == 0, axis=1)
            est = PCR().fit(X[:, donors], y) # find coefficients to donor units using PCR
            Beta[k, t - T0, donors] = est.coef_ # store the unit weights in the matrix Beta
    return Beta

# The matrix Beta will be of shape (n, K, T - T0, n). Each entry (i, k, t, :)
# will contain the donor weights with target unit i, among donors which received
# treatment k, as their first treatment and at period t.
Beta = np.array(Parallel(n_jobs=-1, verbose=3)(delayed(donor_weights)(i) for i in range(n)))

In [ ]:
Beta.shape

In [ ]:
from sklearn.preprocessing import OneHotEncoder

base = np.zeros((n, T)) # baseline response for each unit and period
blip = np.zeros((n, T, lags, K)) # blip effects for each unit i, period t, lag ell and action k
for t in np.arange(T0, T):
    # for each post intervention period and unit estimate the mean baseline response
    base[:, t] = Beta[:, 0, t - T0, :] @ Zobs[:, t] # sum_{j\in I_t^0} \beta_j^{i, I_t^0} Y_{j, t}
    for ell in range(lags):  # we construct the blip effect for lag ell
        for k in range(K):   # and for each action k, i.e. gamma_{j, t, t-ell}(k)
            # we build the obesrve blip effects; we will actually for a moment pretend that
            # every unit is in the I_{t-ell}^k, but then all the "wrong" entries will be corrected
            # by taking the inner product with the donor entries and since donor weights will only
            # be supported on elements in I_{t-ell}^k
            blip[:, t, ell, k] = Zobs[:, t - ell] - base[:, t] # we subtract the baseline response Y_{j, t} - b_{j, t}
            for ellp in range(ell): # for each smaller lag, i.e. period t - ellp, with ellp < ell
                # we subtract the blip effect of the treatment that each unit received at period t - ellp
                # this subtracts gamma_{j, t, t - ellp}(A_{j, t - ell})
                lagDohe = OneHotEncoder(sparse=False).fit_transform(D[:, [t - ellp]]) # this is the treatment at t-ellp
                blip[:, t, ell, k] -= np.sum(blip[:, t, ellp, :] * lagDohe, axis=1)
            # now that we have constructed the observed blip effects for all donor units
            # we can impute the blip effects for all units, using the donor weights
            # we will in fact even replace the blip effects of the donor units, with their
            # corresponding averages, which will induce variance reduction
            blip[:, t, ell, k] = Beta[:, k, t - ell - T0, :] @ blip[:, t, ell, k]

In [ ]:
Dtarget = np.zeros(T - T0)
Dtarget[-2:] = 2

In [ ]:
Wtarget = 0
for ell in range(lags): # for each lag period
    for k in range(K):
        Wtarget += W[0, -1, ell, k] * (Dtarget[-1 - ell] == k) # we add the lag blip effect of that lag treatment

In [ ]:
Wtarget

In [ ]:
Wtarget = base[0, -1]
for ell in range(lags): # for each lag period
    for k in range(K):
        Wtarget += blip[0, -1, ell, k] * (Dtarget[-1 - ell] == k) # we add the lag blip effect of that lag treatment

In [ ]:
Wtarget